In [101]:
from pymongo import MongoClient
import pymongo
import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [102]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())

['admin', 'config', 'local', 'mydb']


In [103]:
db = client['mydb'].xy
result= db.find().sort([("date", pymongo.DESCENDING)]).limit(2)

data = list(result)[0]
start_date = data["date"] - datetime.timedelta(seconds=5)
seq = list(db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}))

In [104]:
list_data = list()
for j in seq:
    del j['date']
    del j['_id']
    
    list_data.append(list(j.values()))

In [105]:
k = int(len(list_data)/30)
list_2 = []
for i in range(1,31):
    list_2.append(list_data[k*i])

In [106]:
list_data = np.array(list_2, dtype=np.float32)
list_data = np.expand_dims(list_data, axis=0)

In [107]:
print(list_data)

[[[  31.166641   -31.23713    -74.375046  -130.99716    223.42577
   -129.42798      7.981476    21.482178     5.4838567  345.59174
    228.90962    216.16376      0.           0.           0.
      0.           0.           0.           0.           0.       ]
  [  31.787346   -31.14447    -73.990166  -138.53969    230.63614
    217.12965     11.052368    42.86789      0.           0.
    230.63614    217.12965      0.           0.           0.
      0.           0.           0.           0.           0.       ]
  [  33.44632    -31.86737    -74.58389   -139.29178    230.34464
    217.57654     13.512695    53.045746     0.           0.
    230.34464    217.57654      0.           0.           0.
      0.           0.           0.           0.           0.       ]
  [  34.257095   -32.780174   -72.396255  -137.65738    229.88402
    219.43549     10.7778015   53.11902      0.           0.
    229.88402    219.43549      0.           0.           0.
      0.           0.           0.  

In [108]:
y_false = np.array([0 for w in range(len(list_data))])
print(list_data.shape)
print(y_false.shape)

(1, 30, 20)
(1,)


In [109]:
# dataloaders
batch_size = 32
device='cuda'

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [110]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)

In [111]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [112]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [113]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)

        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
        
    print(np.mean(acc_list))

In [114]:
tmp_data = TensorDataset(torch.from_numpy(list_data), torch.from_numpy(y_false))
tmp_loader = DataLoader(tmp_data, shuffle=False, batch_size=batch_size)

In [115]:
predict(model, tmp_loader)

0.03125


In [116]:
tmp_data[0]

(tensor([[  31.1666,  -31.2371,  -74.3750, -130.9972,  223.4258, -129.4280,
             7.9815,   21.4822,    5.4839,  345.5917,  228.9096,  216.1638,
             0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000],
         [  31.7873,  -31.1445,  -73.9902, -138.5397,  230.6361,  217.1297,
            11.0524,   42.8679,    0.0000,    0.0000,  230.6361,  217.1297,
             0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000],
         [  33.4463,  -31.8674,  -74.5839, -139.2918,  230.3446,  217.5765,
            13.5127,   53.0457,    0.0000,    0.0000,  230.3446,  217.5765,
             0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
             0.0000,    0.0000],
         [  34.2571,  -32.7802,  -72.3963, -137.6574,  229.8840,  219.4355,
            10.7778,   53.1190,    0.0000,    0.0000,  229.8840,  219.4355,
             0.0000,    0.0000,    0.0000,    0.0000,    0.0000, 